# Rock Melody Transformer Bulk Generator (ver. 1.0)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

WARNING: This complete implementation is a functioning model of the Artificial Intelligence. Please excercise great humility, care, and respect. https://www.nscai.gov/

***

#### Project Los Angeles

#### Tegridy Code 2024

***

# (GPU CHECK)

In [ ]:
#@title NVIDIA GPU check
!nvidia-smi

# (SETUP ENVIRONMENT)

In [ ]:
#@title Install dependencies
!git clone --depth 1 https://github.com/asigalov61/Rock-Melody-Transformer
!pip install huggingface_hub
!pip install einops
!pip install torch-summary
!apt install fluidsynth #Pip does not work for some reason. Only apt works

In [ ]:
#@title Import modules

print('=' * 70)
print('Loading core Rock Melody Transformer modules...')

import os
import pickle
import secrets
import statistics
from time import time
import datetime
import tqdm

import locale
locale.getpreferredencoding = lambda: "UTF-8"

print('=' * 70)
print('Creating I/O dirs...')

if not os.path.exists('/content/Output/Improvs'):
    os.makedirs('/content/Output/Improvs')

if not os.path.exists('/content/Output/Continuations'):
    os.makedirs('/content/Output/Continuations')

print('=' * 70)
print('Loading main Rock Melody Transformer modules...')

import torch

%cd /content/Rock-Melody-Transformer

import TMIDIX

from midi_to_colab_audio import midi_to_colab_audio

from x_transformer_1_23_2 import *

import random

%cd /content/
print('=' * 70)
print('Loading aux Rock Melody Transformer modules...')

import matplotlib.pyplot as plt

from torchsummary import summary
from sklearn import metrics

from IPython.display import Audio, display

from huggingface_hub import hf_hub_download

from google.colab import files

print('=' * 70)
print('Done!')
print('Enjoy! :)')
print('=' * 70)

# (LOAD MODEL)

In [ ]:
#@title Load Rock Melody Transformer Pre-Trained Model

#@markdown Model precision option

model_precision = "bfloat16" # @param ["bfloat16", "float16"]

#@markdown bfloat16 == Half precision/faster speed (if supported, otherwise the model will default to float16)

#@markdown float16 == Full precision/fast speed

plot_tokens_embeddings = False # @param {type:"boolean"}

print('=' * 70)
print('Loading Rock Melody Transformer Pre-Trained Model...')
print('Please wait...')
print('=' * 70)

full_path_to_models_dir = "/content/Rock-Melody-Transformer/Model"

model_checkpoint_file_name = 'Rock_Melody_Transformer_Medium_Trained_Model_21749_steps_0.7973_loss_0.7597_acc.pth'
model_path = full_path_to_models_dir+'/'+model_checkpoint_file_name
if os.path.isfile(model_path):
  print('Model already exists...')

else:
  hf_hub_download(repo_id='asigalov61/Rock-Melody-Transformer',
                  filename=model_checkpoint_file_name,
                  local_dir='/content/Rock-Melody-Transformer/Model',
                  local_dir_use_symlinks=False)

print('=' * 70)
print('Instantiating model...')

device_type = 'cuda'

if model_precision == 'bfloat16' and torch.cuda.is_bf16_supported():
  dtype = 'bfloat16'
else:
  dtype = 'float16'

if model_precision == 'float16':
  dtype = 'float16'

ptdtype = {'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = torch.amp.autocast(device_type=device_type, dtype=ptdtype)

SEQ_LEN = 4096 # Models seq len
PAD_IDX = 2081 # Models pad index

# instantiate the model

model = TransformerWrapper(
    num_tokens = PAD_IDX+1,
    max_seq_len = SEQ_LEN,
    attn_layers = Decoder(dim = 1024, depth = 24, heads = 16, attn_flash = True)
    )

model = AutoregressiveWrapper(model, ignore_index = PAD_IDX, pad_value=PAD_IDX)

model.cuda()
print('=' * 70)

print('Loading model checkpoint...')

model.load_state_dict(torch.load(model_path))
print('=' * 70)

model.eval()

print('Done!')
print('=' * 70)

print('Model will use', dtype, 'precision...')
print('=' * 70)

# Model stats
print('Model summary...')
summary(model)

# Plot Token Embeddings
if plot_tokens_embeddings:
  tok_emb = model.net.token_emb.emb.weight.detach().cpu().tolist()

  cos_sim = metrics.pairwise_distances(
    tok_emb, metric='cosine'
  )
  plt.figure(figsize=(7, 7))
  plt.imshow(cos_sim, cmap="inferno", interpolation="nearest")
  im_ratio = cos_sim.shape[0] / cos_sim.shape[1]
  plt.colorbar(fraction=0.046 * im_ratio, pad=0.04)
  plt.xlabel("Position")
  plt.ylabel("Position")
  plt.tight_layout()
  plt.plot()
  plt.savefig("/content/Rock-Melody-Transformer-Tokens-Embeddings-Plot.png", bbox_inches="tight")

# (GENERATE)

# (SETUP MODEL CHANNELS MIDI PATCHES)

In [ ]:
# @title Setup and load model channels MIDI patches

model_channel_0_piano_family = "Acoustic Grand" # @param ["Acoustic Grand", "Bright Acoustic", "Electric Grand", "Honky-Tonk", "Electric Piano 1", "Electric Piano 2", "Harpsichord", "Clav"]
model_channel_1_chromatic_percussion_family = "Music Box" # @param ["Celesta", "Glockenspiel", "Music Box", "Vibraphone", "Marimba", "Xylophone", "Tubular Bells", "Dulcimer"]
model_channel_2_organ_family = "Church Organ" # @param ["Drawbar Organ", "Percussive Organ", "Rock Organ", "Church Organ", "Reed Organ", "Accordion", "Harmonica", "Tango Accordion"]
model_channel_3_guitar_family = "Acoustic Guitar(nylon)" # @param ["Acoustic Guitar(nylon)", "Acoustic Guitar(steel)", "Electric Guitar(jazz)", "Electric Guitar(clean)", "Electric Guitar(muted)", "Overdriven Guitar", "Distortion Guitar", "Guitar Harmonics"]
model_channel_4_bass_family = "Fretless Bass" # @param ["Acoustic Bass", "Electric Bass(finger)", "Electric Bass(pick)", "Fretless Bass", "Slap Bass 1", "Slap Bass 2", "Synth Bass 1", "Synth Bass 2"]
model_channel_5_strings_family = "Violin" # @param ["Violin", "Viola", "Cello", "Contrabass", "Tremolo Strings", "Pizzicato Strings", "Orchestral Harp", "Timpani"]
model_channel_6_ensemble_family = "Choir Aahs" # @param ["String Ensemble 1", "String Ensemble 2", "SynthStrings 1", "SynthStrings 2", "Choir Aahs", "Voice Oohs", "Synth Voice", "Orchestra Hit"]
model_channel_7_brass_family = "Trumpet" # @param ["Trumpet", "Trombone", "Tuba", "Muted Trumpet", "French Horn", "Brass Section", "SynthBrass 1", "SynthBrass 2"]
model_channel_8_reed_family = "Alto Sax" # @param ["Soprano Sax", "Alto Sax", "Tenor Sax", "Baritone Sax", "Oboe", "English Horn", "Bassoon", "Clarinet"]
model_channel_9_pipe_family = "Flute" # @param ["Piccolo", "Flute", "Recorder", "Pan Flute", "Blown Bottle", "Skakuhachi", "Whistle", "Ocarina"]
model_channel_10_drums_family = "Drums" # @param ["Drums"]

print('=' * 70)
print('Setting up patches...')
print('=' * 70)

instruments = [v[1] for v in TMIDIX.Number2patch.items()]

patches = [instruments.index(model_channel_0_piano_family),
                       instruments.index(model_channel_1_chromatic_percussion_family),
                       instruments.index(model_channel_2_organ_family),
                       instruments.index(model_channel_3_guitar_family),
                       instruments.index(model_channel_4_bass_family),
                       instruments.index(model_channel_5_strings_family),
                       instruments.index(model_channel_6_ensemble_family),
                       instruments.index(model_channel_7_brass_family),
                       instruments.index(model_channel_8_reed_family),
                       9, # Drums patch
                       instruments.index(model_channel_9_pipe_family),
                       ] + [0, 0, 0, 0, 0]

print('Done!')
print('=' * 70)

# (BULK IMPROV)

In [ ]:
#@title Bulk Improv Generator

#@markdown NOTE: You can stop bulk generation at any time

#@markdown Improv type

improv_type = "Random Freestyle" # @param ["Random Freestyle", "Custom"]

#@markdown Custom Improv settings

first_chord_semitone = 0 # @param {type:"slider", min:0, max:11, step:1}

#@markdown Generation settings

number_of_tokens_to_generate = 2048 # @param {type:"slider", min:30, max:4096, step:4}
number_of_generation_cycles = 10 # @param {type:"slider", min:1, max:32, step:1}
number_of_batches_to_generate = 16 # @param {type:"slider", min:1, max:32, step:1}
temperature = 0.9 # @param {type:"slider", min:0.1, max:1, step:0.05}

#@markdown Other settings

verbose = False # @param {type:"boolean"}

date_time = datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d_%H-%M-%S')

if improv_type == 'Random Freestyle':

  outy = [random.randint(12, 24)+2056, 0]

if improv_type == 'Custom':

  outy = [first_chord_semitone+12+2056, 0]

  improv_type += '_ST'+str(first_chord_semitone)

output_dir = '/content/Output/Improvs/'+improv_type+'_'+str(date_time) + '/'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print('=' * 70)
print('Rock Melody Transformer Bulk Improv Model Generator')
print('=' * 70)
print('Generating', number_of_generation_cycles * number_of_batches_to_generate, 'compositions', number_of_tokens_to_generate, 'tokens each')
print('=' * 70)
print('Total number of tokens to generate:', number_of_generation_cycles*number_of_batches_to_generate * number_of_tokens_to_generate)
print('=' * 70)
print('Output directory:', output_dir)
print('=' * 70)
print('Generating...Please wait...')
print('=' * 70)

try:

  for gc in tqdm.tqdm(range(number_of_generation_cycles)):

    if verbose:
      print('Selected Improv sequence:')
      print(outy)
      print('=' * 70)

    torch.cuda.empty_cache()

    inp = [outy] * number_of_batches_to_generate

    inp = torch.LongTensor(inp).cuda()

    with ctx:
      out = model.generate(inp,
                            number_of_tokens_to_generate,
                            temperature=temperature,
                            return_prime=True,
                            verbose=verbose)

    out0 = out.tolist()

    if verbose:
      print('=' * 70)
      print('Done!')
      print('=' * 70)

    torch.cuda.empty_cache()

    #======================================================================

    if verbose:
      print('Rendering results...')

    for i in range(number_of_batches_to_generate):

      if verbose:
        print('=' * 70)
        print('Batch #', i)
        print('=' * 70)

      out1 = out0[i]

      if verbose:
        print('Sample INTs', out1[:12])
        print('=' * 70)

      if len(out1) != 0:

          song = out1
          song_f = []

          time = 0
          dur = 32
          channel = 0
          pitch = 60
          vel = 90

          for ss in song:

              if 0 <= ss < 256:

                  time += ss * 16

              if 256 < ss < 512:

                  dur =  (ss-256) * 16

              if 512 < ss < 2048:

                  chan = (ss-512) // 128

                  if chan == 11:
                    channel = 9
                  else:
                    if chan > 8:
                      channel = chan + 1
                    else:
                      channel = chan

                  if channel == 9:
                    patch = 128
                  else:
                    patch = channel * 8

                  pitch = (ss-512) % 128

              if 2048 < ss < 2056:
                  vel = ((ss - 2048)+1) * 15

                  song_f.append(['note', time, dur, channel, pitch, vel, patch])

          stats = TMIDIX.Tegridy_ms_SONG_to_MIDI_Converter(song_f,
                                                            output_signature = 'Rock Melody Transformer',
                                                            output_file_name = output_dir + 'Rock-Melody-Transformer-Composition_'+str(i+(gc*number_of_batches_to_generate)).zfill(5),
                                                            track_name='Project Los Angeles',
                                                            list_of_MIDI_patches=patches,
                                                            verbose=verbose)

          if verbose:
            print('=' * 70)

except KeyboardInterrupt:
  print('STOP!!!')
  print('=' * 70)
  print('Stopping generation...')

print('Done!')
print('=' * 70)

# (ZIP AND DOWNLOAD RESULTS)

In [ ]:
#@title Zip and download all bulk generations results

print('=' * 70)

try:
    os.remove('Rock_Melody_Transformer_Bulk_Generator_Output.zip')
except OSError:
    pass

print('Zipping... Please wait...')
print('=' * 70)

%cd /content/Output/
!zip -r Rock_Melody_Transformer_Bulk_Generator_Output.zip *
%cd /content/

print('=' * 70)
print('Done!')
print('=' * 70)

print('Downloading final zip file...')
print('=' * 70)

files.download('/content/Output/Rock_Melody_Transformer_Bulk_Generator_Output.zip')

print('Done!')
print('=' * 70)

In [ ]:
# @title Delete results directory and files

#@markdown WARNING: This can't be undone so make sure you downloaded the results first

print('=' * 70)
print('Deleting... Please wait...')
print('=' * 70)

!rm -rf /content/Output
print('Done!')
print('=' * 70)

# Congrats! You did it! :)